In [40]:
# Import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import dataset
dataset = pd.read_csv("C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\Datasets\\iris.csv")
seed = 7
np.random.seed(seed)

In [41]:
# veri setine bakış
dataset.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [42]:
# Veri setinde sütun isimleri yokmuş. Hadi ekleyelim.
iris_cols = ['sepal-length', 'sepal-with', 'pedal-length','pedal-width','label'] 
dataset.columns = iris_cols

# Doğru eklemiş miyiz bakalım:
dataset.head()

,sepal-length,sepal-with,pedal-length,pedal-width,label
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [43]:
# Bağımlı ve bağımsız değişkenleri ayırma

# Python'da sütunların indeks değeri 0'dan başladığından ve 5 sütun olduğundan son niteliğin indeks değeri 4 olacaktır. 
# iloc ile sütun seçerken ilk değer dahil ikinci değer hariçtir. bu sebeple 0'ı dahil 4'ü hariç tutacağız. 
# Çünkü son indeks değeri label yani hedef değişkene ait. 
# iloc[] içindeki İlk iki nokta (:) tüm satırları ifade ederken "," sonrası sütunlardan seçilecek filtreyi ifade eder.
# 0:4 demek 0 dahil 4 hariç indeksli sütunları seç demek. Yani 0,1,2,3

X = dataset.iloc[:,0:4].values

# Hedef niteliğimiz 4. indekste idi. O sebeple : ile tüm satırları "," den sonra 4 ile hedef niteliğin indeksini seçiyoruz.
y = dataset.iloc[:,4].values

In [44]:
# Hedef Değişkeni Dönüştürmek
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

y = np_utils.to_categorical(y)

In [45]:
# Eğitim ve Test Seti olarak Ayırmak
from sklearn.model_selection import train_test_split

#Sıralamaya dikkat. Aynı sonuçlar için random_state değeri aynı olmalıdır.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [46]:
# Nitelikler matrisini (X) StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
# Keras kütüphanelerini indirme
import keras
from keras.models import Sequential
from keras.layers import Dense

In [48]:
# YAPAY SİNİR AĞI NESNESİ OLUŞTURMA
# Yapay sinir ağı iki farklı şekilde başlatılabilir: 1. Katmanlar dizilimi (Sequantial layers) olarak. 2. Graph olarak.
# Birinci yolda sadece yukarıda indirilen Sequantial sınıfından nesne yaratılır. Bu da Karar ağacı, SVM gibi bir sınıflandırıcı olacak.
classifier = Sequential()

# Bir Sinir Ağını Stochastic Gradient Descent ile Eğitmenin Aşamaları
1. Sıfıra yakın rastgele değerler ile ağırlık katsayılarını belirle.
2. Input layera ilk gözlemi(bir satır) her bir düğüme bir nitelik (nitelikler matrisi sütün sayısı kadar) düşecek şekilde ver.
3. Ağı çalıştır ve ilk tahmin y değerini üret.
4. Üretilen y ile tahmin edilen y arasındaki hatayı hesapla.
5. Hesaplanan hatayı geri besleme olarak gönder ve her bir ağırlık katsayısını güncelle.
6. 1 ve 5 arasını ya her bir satırdan sonra veya belli bir satır sayısından (grup-batch) sonra tekrarla.
7. Veri setindeki tüm satırların bitmesi bir epoch oluşturur. Bunu defalarca tekrarla.

# GİRDİ VE İLK GİZLİ KATMANI EKLEME
 classifier nesnesinin add() metodu içine Dense() Sınıfının nesnesnesini parametre olarak veriyoruz.
 Dense içindeki ilk parametre 
 kernel_initializer (eski init): Başlangıç ağırlıklarını belirler. uniform fonksiyonunu kullanıyoruz.
 input_dim: Yapay sinir ağı nesnesine ilk katman ekleme işlemi yapılırken mutlaka girdi katmanında kaç düğüm olacağı bildirilir.
 aksi halde ilk gizli katman kendisine kaç düğümden bağlantı olacak bilemez. 
 Bu rakam nitelikler matrisindeki sütun sayısıdır yani 4

 units (eski:output_dim): Bu sayının nasıl verileceğine dair sağlam bir kural yok. Biraz sanatkarlık ve tecrübe
 gerektiriyor. Tecrübeye dayalı ipucu: Girdi ve çıktı düğüm sayısının ortalamasını verebilirsin. 
 Bu örnekte (4+3=7/2=3,5 ~ 4) Ya da parametre tuning tekniklerini kullanabilirsin.

 activation: Gizli düğümler için rectifier çıktı düğüm için softsign olsun. Rectifier action parametresinde relu olarak tanımlı


In [49]:
classifier.add(Dense(kernel_initializer = 'uniform', input_dim = 4, units = 4,  activation = 'relu'))

# İKİNCİ GİZLİ KATMANI EKLEME

 İkinci Gizli Katmanı Eklemek

 Yukarıda ilk gizli katman ve düğüm sayısı belli olduğu için burada input_dim parametresi kullanmaya gerek yok.
 Gizli katman sayısını (units) aynı tutuyoruz. kaba formül kullandık yine.

In [50]:
#classifier.add(Dense(kernel_initializer = 'uniform', units = 4,  activation = 'relu'))

# ÇIKTI KATMANI (OUTPUT LAYER) EKLEME
 Bu katmanın bir öncekinden farkı düğüm sayısı ve activasyon fonksiyonu olacak. Hedef değişken üç farklı değer alacağı için units=3

In [51]:
classifier.add(Dense(kernel_initializer = 'uniform', units = 3,  activation = 'softmax'))

In [52]:
# optimizer: Stochastic Gradient Descent'i temsilen adam
# loss: SGD'ni optimizasyonu için kullanılacak loss function. Tahmin y ile gerçek ye değeri arasını hesaplayıp en optimal değeri
# SGD'ye buldurur. Logaritmik loss function. Binary olduğu için categorical_crossentropy kullanıyoruz.
# metrics: İlave olarak burada model değerlendirme kriterleri belirlenir. Bir liste halinde verilir. 
# Biz şimdilik sadece accuracy veriyoruz.
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
classifier.fit(X_train, y_train, batch_size=5, epochs=100)

Epoch 1/100
119/119 [==============================] - 0s 3ms/step - loss: 1.0985 - acc: 0.3445
Epoch 2/100
119/119 [==============================] - 0s 320us/step - loss: 1.0952 - acc: 0.3697
Epoch 3/100
119/119 [==============================] - 0s 294us/step - loss: 1.0855 - acc: 0.5966
Epoch 4/100
119/119 [==============================] - 0s 277us/step - loss: 1.0663 - acc: 0.6807
Epoch 5/100
119/119 [==============================] - 0s 256us/step - loss: 1.0374 - acc: 0.6639
Epoch 6/100
119/119 [==============================] - 0s 252us/step - loss: 1.0000 - acc: 0.6639
Epoch 7/100
119/119 [==============================] - 0s 252us/step - loss: 0.9565 - acc: 0.6639
Epoch 8/100
119/119 [==============================] - 0s 227us/step - loss: 0.9107 - acc: 0.6639
Epoch 9/100
119/119 [==============================] - 0s 324us/step - loss: 0.8618 - acc: 0.6639
Epoch 10/100
119/119 [==============================] - 0s 294us/step - loss: 0.8155 - acc: 0.6639
Epoch 11/100
119/119 

119/119 [==============================] - 0s 294us/step - loss: 0.1911 - acc: 0.9496
Epoch 84/100
119/119 [==============================] - 0s 341us/step - loss: 0.1883 - acc: 0.9496
Epoch 85/100
119/119 [==============================] - 0s 345us/step - loss: 0.1857 - acc: 0.9580
Epoch 86/100
119/119 [==============================] - 0s 324us/step - loss: 0.1831 - acc: 0.9580
Epoch 87/100
119/119 [==============================] - 0s 261us/step - loss: 0.1803 - acc: 0.9580
Epoch 88/100
119/119 [==============================] - 0s 231us/step - loss: 0.1778 - acc: 0.9580
Epoch 89/100
119/119 [==============================] - 0s 299us/step - loss: 0.1755 - acc: 0.9580
Epoch 90/100
119/119 [==============================] - 0s 265us/step - loss: 0.1727 - acc: 0.9580
Epoch 91/100
119/119 [==============================] - 0s 273us/step - loss: 0.1703 - acc: 0.9580
Epoch 92/100
119/119 [==============================] - 0s 349us/step - loss: 0.1680 - acc: 0.9580
Epoch 93/100
119/119 [=

In [54]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred.round(), normalize=True)

0.90000000000000002